## Prompt
Imaginemos que tenemos un cliente que quiere que se procesen 501 peticiones al mismo tiempo sobre diversa información de stock.  Las peticiones de información se procesan en nuestras máquinas utilizando un recurso limitado llamado procesador.  El procesador puede procesar 10 peticiones al mismo tiempo.  Cada máquina tiene 5 procesadores.  Si la solicitud de información se realiza después de las 12:00, se considera hora punta y es probable que la solicitud compita por los recursos.  Por ello, el cliente necesitará una asignación adicional del 25%.

Esta pregunta requiere varias conversiones de unidades, de petición a procesadores y a máquinas, y luego hay que tener en cuenta el tiempo.

In [ ]:
requests = 501 #solicitas

In [ ]:
def convert_request_to_processors(requests):
    return requests / 10

In [ ]:
convert_request_to_processors(requests)

No podemos tener la mitad de un procesador, o se tiene o no se tiene.  Deberíamos redondear

In [ ]:
import math

In [ ]:
def convert_request_to_processors(request):
    return math.ceil(request/10)

In [ ]:
convert_request_to_processors(requests)

Ahora debemos convertir los procesadores en máquinas

In [ ]:
def convert_processors_to_machines(processors):
    return math.ceil(processors / 5)

In [ ]:
convert_processors_to_machines(51)

Si la solicitud se realiza antes de las 12:00, necesitaremos 11 máquinas para procesar 501 solicitudes de información sobre existencias al mismo tiempo.  Si la solicitud se realiza después de las 12.00 horas, debemos tener en cuenta que se necesita un 25% más de capacidad

In [ ]:
import datetime

In [ ]:
def apply_additional_capacity(machines, time):
    if time >= datetime.time(12, 0):
        additional_capacity = math.ceil(machines * .25)
        machines += additional_capacity
    return machines

In [ ]:
apply_additional_capacity(11, datetime.time(12,1))

Ahora vamos a crear la interfaz de usuario para estos cálculos básicos, utilizando el panel https://panel.holoviz.org/  

In [ ]:
import panel as pn
import datetime
pn.extension()

¿Cuáles son nuestras entradas? Peticiones y hora del día. 

In [ ]:
request_input = pn.widgets.IntInput(name='Requests', value=1, step=1, start=1, end=1000000)
time_input = pn.widgets.DatetimeInput(name='Time of request', value=datetime.datetime(2019, 2, 8, 0,0), format ='%H:%M') # requires datetime package
app = pn.Row(request_input,time_input)
app

Ahora querremos una función que procese nuestro cálculo, un botón que dispare esa función y una salida para el cálculo

In [ ]:
machines_output = pn.widgets.IntInput(name='Machines', value = 0, disabled= True)

In [ ]:
def on_click_calculate(event):
    processors = convert_request_to_processors(request_input.value)
    machines = convert_processors_to_machines(processors)
    if time_input.value.time() >= datetime.time(12, 0):
        additional_capacity = peak_time_capacity(machines)
        machines += additional_capacity
    machines_output.value = machines

In [ ]:
calculate_button = pn.widgets.Button(name='Calculate', button_type='primary')
calculate_button.on_click(on_click_calculate)

In [ ]:
app = pn.Column(pn.Row(request_input,time_input), calculate_button, machines_output)

**NOTA:** He tenido problemas con el botón ui en jupyter lab, si te encuentras con esto también ejecutar en jupyter classic. <br>
Hice esto yendo a  Help > Launch Classic Notebook <br>
![image.png](images/launch_classic.png)

In [ ]:
app.servable()

En este punto tenemos esencialmente el esqueleto de nuestra calculadora, conocemos nuestras entradas y salidas y hemos definido funciones para la lógica de la calculadora